In [3]:
import numpy as np
import tifffile
from scipy import ndimage
from skimage.filters import threshold_otsu, gaussian
from skimage.morphology import binary_closing, disk
import os
import glob
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/Static-A-2/Cadherins'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-2/Membrane'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Find all Cadherin .tif files
cadherin_files = glob.glob(os.path.join(input_dir, '*Cadherins.tif'))
print(f"Found {len(cadherin_files)} Cadherin files to process")

# Process each file
for cadherin_file in cadherin_files:
    # Get base filename for output
    filename = os.path.basename(cadherin_file)
    base_name = filename.replace('_Cadherins.tif', '')

    print(f"Processing: {filename}")

    # Load the image
    cadherin_img = tifffile.imread(cadherin_file)
    print(f"  Image shape: {cadherin_img.shape}")

    # Extract Cadherin channel if needed
    if len(cadherin_img.shape) == 2:
        # Single channel image (already Cadherin)
        print("  Detected single-channel Cadherin image")
        membrane_channel = cadherin_img
    elif len(cadherin_img.shape) == 3 and cadherin_img.shape[0] == 3:
        # Format is (C, H, W)
        print("  Detected format: (C, H, W)")
        membrane_channel = cadherin_img[0]  # First channel
    elif len(cadherin_img.shape) == 3 and cadherin_img.shape[2] == 3:
        # Format is (H, W, C)
        print("  Detected format: (H, W, C)")
        membrane_channel = cadherin_img[:, :, 0]  # First channel
    else:
        print(f"  Unexpected image shape: {cadherin_img.shape}. Using first channel/plane.")
        if len(cadherin_img.shape) == 3:
            membrane_channel = cadherin_img[0] if cadherin_img.shape[0] < cadherin_img.shape[1] else cadherin_img[:, :, 0]
        else:
            membrane_channel = cadherin_img

    # Apply Gaussian blur to reduce noise
    print("  Applying Gaussian blur...")
    membrane_smoothed = gaussian(membrane_channel, sigma=1)

    # Apply Otsu thresholding
    print("  Performing Otsu thresholding...")
    thresh = threshold_otsu(membrane_smoothed)
    binary_membrane = membrane_smoothed > thresh

    # Apply binary closing with larger radius to fill holes
    print("  Applying binary closing...")
    selem_close = disk(3)  # Increased radius to fill more holes
    closed_membrane = binary_closing(binary_membrane, selem_close)

    # Apply erosion to remove small isolated elements
    print("  Applying erosion to remove small elements...")
    selem_erode = disk(1)  # Small radius for erosion
    final_membrane = ndimage.binary_erosion(closed_membrane, structure=selem_erode)

    # Save only the membrane mask
    membrane_mask_path = os.path.join(output_dir, f"{base_name}_membrane_mask.tif")
    tifffile.imwrite(membrane_mask_path, final_membrane.astype(np.uint8))
    print(f"  Saved membrane mask to {membrane_mask_path}")

    # Print statistics
    print(f"  Threshold value used: {thresh}")
    print(f"  Percentage of pixels identified as membrane: {(final_membrane.sum() / final_membrane.size * 100):.2f}%")
    print(f"  Processing complete for {filename}\n")

print("All processing complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 18 Cadherin files to process
Processing: denoised_0Pa_A1_20dec21_20xA_L2RA_FlatA_seq001_contrast_Cadherins.tif
  Image shape: (1024, 1024)
  Detected single-channel Cadherin image
  Applying Gaussian blur...
  Performing Otsu thresholding...
  Applying binary closing...
  Applying erosion to remove small elements...
  Saved membrane mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-2/Membrane/denoised_0Pa_A1_20dec21_20xA_L2RA_FlatA_seq001_contrast_membrane_mask.tif
  Threshold value used: 0.43546855488668756
  Percentage of pixels identified as membrane: 34.97%
  Processing complete for denoised_0Pa_A1_20dec21_20xA_L2RA_FlatA_seq001_contrast_Cadherins.tif

Processing: denoised_0Pa_A1_20dec21_20xA_L2RA_FlatA_seq002_contrast_Cadherins.tif
  Image shape: (1024, 1024)
  Detected single-channel Cadherin image
  Applying Gau